<a href="https://colab.research.google.com/github/SahithiChakrapani/SQL_Optimization_Project/blob/main/SQL_Optimization_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%sql


-- find latest salary for each employee
with CTE as(select *, rank() over(partition by employee_id order by change_date desc) as latest_salary_rank,
            rank() over(partition by employee_id order by change_date Asc) as first_salary_rank
from salary_history),
Latest_salary_CTE as (
select employee_id,salary as latest_Salary
from CTE
where latest_salary_rank = 1),

-- calculate total number of promotions each employee has received
Promotions_CTE as (select employee_id,count(*) as promotion_count
from CTE
where promotion = 'Yes'
group by employee_id),

-- dertermine the maximum salary hike percentage between any two consecutive salary changes for each employee
Previous_Salary_cte as (select *,
                        lead(salary,1) over (partition by employee_id order by change_date desc) as previous_Salary,
salary as Current_Salary,
                        lead(change_date,1) over (partition by employee_id order by change_date desc) as previous_Change_date
from CTE),
Salary_Growth_CTE as (
select employee_id, max(round(100.0*(salary-previous_Salary)/previous_Salary,2)) as Employee_Max_Salary_Hike
from Previous_Salary_cte
group by employee_id
),

-- find employees whos salary has never decreased overtime
Salary_NeverDecreased as (
select DISTINCT employee_id, 'N' as never_decreased
  from Previous_Salary_cte
  where Current_Salary < previous_Salary

),

-- Avg months between changes
Avg_months_cte as (SELECT employee_id,
   avg((CAST(strftime('%Y', change_date) AS INT) - CAST(strftime('%Y', previous_Change_date) AS INT)) * 12 +
   (CAST(strftime('%m', change_date) AS INT) - CAST(strftime('%m', previous_Change_date) AS INT)))
   AS Avg_diff_months
from Previous_Salary_cte
group by employee_id),

-- rank employees by their salary growth rate from first to last recorderd salary), breaking ties by earliest join date
growth_rate_cte as (select employee_id, round(100.0*(max(case when latest_salary_rank = 1 then salary end)-
max(case when first_salary_rank = 1 then salary end))/max(case when first_salary_rank = 1 then salary end),2) as emp_growth_rate,
                   min(change_date) as joining_date
from cte
group by employee_id),

Salary_growth_rank_cte as (
select *,rank() over (order by emp_growth_rate desc,joining_date asc) as growth_rate_rank
from growth_rate_cte)

--- combine all metrix and show in single table output

select E.employee_id,E.name,LS.latest_Salary,IFNULL(P.promotion_count,0) as No_of_Promotions,SG.Employee_Max_Salary_Hike,
ifnull(SD.never_decreased,'Y') as Never_decreased,AM.Avg_diff_months,GR.growth_rate_rank
from employees as E
left join Latest_salary_CTE as LS on LS.employee_id = E.employee_id
left join Promotions_CTE as P on P.employee_id = E.employee_id
left join Salary_Growth_CTE as SG on SG.employee_id = E.employee_id
left join Salary_NeverDecreased as SD on SD.employee_id = E.employee_id
left join Avg_months_cte as AM on AM.employee_id = E.employee_id
left join Salary_growth_rank_cte as GR on GR.employee_id = E.employee_id



 * sqlite:///employees.db
Done.


employee_id,name,latest_Salary,No_of_Promotions,Employee_Max_Salary_Hike,Never_decreased,Avg_diff_months,growth_rate_rank
1,Alice,70000,1,27.27,Y,16.0,2
2,Bob,68000,2,30.77,Y,23.5,1
3,Charlie,72000,1,10.77,Y,28.5,5
4,David,49000,0,8.89,Y,18.0,6
5,Eve,75000,2,20.97,Y,31.0,4
6,Sumit,75000,1,87.5,N,21.0,3


In [ ]:
%%sql
-- find latest salary for each employee
with CTE as(select *, rank() over(partition by employee_id order by change_date desc) as latest_salary_rank,
            rank() over(partition by employee_id order by change_date Asc) as first_salary_rank,
            lead(salary,1) over (partition by employee_id order by change_date desc) as previous_Salary,
            lead(change_date,1) over (partition by employee_id order by change_date desc) as previous_Change_date
from salary_history),


-- rank employees by their salary growth rate from first to last recorderd salary), breaking ties by earliest join date
growth_rate_cte as (select employee_id, round(100.0*(max(case when latest_salary_rank = 1 then salary end)-
max(case when first_salary_rank = 1 then salary end))/max(case when first_salary_rank = 1 then salary end),2) as emp_growth_rate,
                   min(change_date) as joining_date
from cte
group by employee_id)


-- optimize code , reduced CTEs
-- latest salary cte
select A.*,B.emp_latest_salary,B.emp_total_promotions,B.Employee_Max_Salary_Hike,B.salary_never_decreased,B.Avg_diff_months,B.growth_rate_rank
from
(select employee_id,name
from employees) as A

left join

(select CTE.employee_id, max(case when latest_salary_rank = 1 then salary end) as emp_latest_salary,
sum(case when promotion = 'Yes' then 1 else 0 end) as emp_total_promotions,
max(round(100.0*(salary-previous_Salary)/previous_Salary,2)) as Employee_Max_Salary_Hike,
ifnull(max(case when salary < previous_Salary then 'N' end),'Y') as  salary_never_decreased,
   avg((CAST(strftime('%Y', change_date) AS INT) - CAST(strftime('%Y', previous_Change_date) AS INT)) * 12 +
   (CAST(strftime('%m', change_date) AS INT) - CAST(strftime('%m', previous_Change_date) AS INT)))
   AS Avg_diff_months,
   rank() over (order by emp_growth_rate desc,joining_date asc) as growth_rate_rank
from CTE
join growth_rate_cte GR on CTE.employee_id = GR.employee_id
group by CTE.employee_id) as B
on A.employee_id = B.employee_id

--select * from employees





 * sqlite:///employees.db
Done.


employee_id,name,emp_latest_salary,emp_total_promotions,Employee_Max_Salary_Hike,salary_never_decreased,Avg_diff_months,growth_rate_rank
1,Alice,70000,1,27.27,Y,16.0,2
2,Bob,68000,2,30.77,Y,23.5,1
3,Charlie,72000,1,10.77,Y,28.5,5
4,David,49000,0,8.89,Y,18.0,6
5,Eve,75000,2,20.97,Y,31.0,4
6,Sumit,75000,1,87.5,N,21.0,3
